### Coding：聚合、合并和重塑数据的方法

> ### **Question**

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

### 2.1 层次化索引：能在一个轴上拥有多个索引级别

In [2]:
data = pd.Series(np.random.randn(9), # 创建一个Series，并用一个由列表或数组组成的列表作为索引
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],\
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [4]:
data['b'] # 部分索引的使用

1   -0.555730
3    1.965781
dtype: float64

In [5]:
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [7]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

In [8]:
data.unstack() # 层次化索引在数据重塑和基于分组的操作  通过unstack方法将这段数据重新安排到一个DataFrame中

1         2         3
a -0.204708  0.478943 -0.519439
b -0.555730       NaN  1.965781
c  1.393406  0.092908       NaN
d       NaN  0.281746  0.769023

In [9]:
data.unstack().stack() # unstack的逆运算stack

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),\ # 每条轴都可以有分层索引
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],\
                     columns=[['Ohio', 'Ohio', 'Colorado'],\
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ['key1', 'key2'] # 各层的名字可以是字符串 也可以是Python对象
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### Note：区分索引名state、color与行标签

In [12]:
frame['Ohio'] # 选取列分组

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

> ### 重排 与 分级排序：

> ### 重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序

In [13]:
frame.swaplevel('key1', 'key2') # swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象，但数据不会发生改变

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [14]:
frame.sort_index(level=1)  # sort_index会根据单个级别中的值对数据进行排序，结果按照指定顺序进行排序
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

> ### 根据级别汇总统计

In [15]:
frame.sum(level='key2')  # level选项用于指定在某条轴上求和的界别  结合pandas的groupby功能

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [16]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

> ### 使用DataFrame的列进行索引

In [17]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

a  b    c  d
0  0  7  one  0
1  1  6  one  1
2  2  5  one  2
3  3  4  two  0
4  4  3  two  1
5  5  2  two  2
6  6  1  two  3

In [18]:
frame2 = frame.set_index(['c', 'd']) # DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame
frame2 # 默认情况，c d列会从DataFrame中移除，通过drop=False其保留下来

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [19]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [20]:
frame2.reset_index()  # reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面

c  d  a  b
0  one  0  0  7
1  one  1  1  6
2  one  2  2  5
3  two  0  3  4
4  two  1  4  3
5  two  2  5  2
6  two  3  6  1

### 2.2 合并数据集

- pandas.merge 可根据一个或多个键将不同DataFrame中的行连接起来 结合数据库的join操作

- pandas.concat 沿着一条轴将多个对象堆叠到一起

- combine_first(实例方法) 可以将重复数据拼接在一起，用一个对象中的值填充另一个对象中的缺失值

In [21]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df1

data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b

In [22]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2 # 多对一的合并

data2 key
0      0   a
1      1   b
2      2   d

In [23]:
pd.merge(df1, df2) # 通过merge对指定列进行连接

data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0

In [24]:
pd.merge(df1, df2, on='key')

data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0

In [25]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],\
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],\
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey') # 对象之间列明不同的情况

data1 lkey  data2 rkey
0      0    b      1    b
1      1    b      1    b
2      6    b      1    b
3      2    a      0    a
4      4    a      0    a
5      5    a      0    a

In [26]:
pd.merge(df1, df2, how='outer') # 默认情况 merge做的是内连接 inner -->结果中的键是交集
# inner 使用两个表都有的键
# left 使用坐标中所有的键
# right 使用右表中所有的键
# outer 使用两个表中所有的键 外连接求取的是键的交集，组合了左右连接的效果

data1 key  data2
0    0.0   b    1.0
1    1.0   b    1.0
2    6.0   b    1.0
3    2.0   a    0.0
4    4.0   a    0.0
5    5.0   a    0.0
6    3.0   c    NaN
7    NaN   d    2.0

In [27]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   b

In [28]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df2

data2 key
0      0   a
1      1   b
2      2   a
3      3   b
4      4   d

In [29]:
pd.merge(df1, df2, on='key', how='left') # 多对多的合并操作  即行的笛卡尔积  
# 连接方式只影响出现在结果中的不同的键的值

data1 key  data2
0       0   b    1.0
1       0   b    3.0
2       1   b    1.0
3       1   b    3.0
4       2   a    0.0
5       2   a    2.0
6       3   c    NaN
7       4   a    0.0
8       4   a    2.0
9       5   b    1.0
10      5   b    3.0

In [30]:
pd.merge(df1, df2, how='inner')

data1 key  data2
0      0   b      1
1      0   b      3
2      1   b      1
3      1   b      3
4      5   b      1
5      5   b      3
6      2   a      0
7      2   a      2
8      4   a      0
9      4   a      2

In [31]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],\
                     'key2': ['one', 'two', 'one'],\
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],\
                      'key2': ['one', 'one', 'one', 'two'],\
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')  # 根据多个键进行合并，传入一个由列名组成的列表即可

key1 key2  lval  rval
0  foo  one   1.0   4.0
1  foo  one   1.0   5.0
2  foo  two   2.0   NaN
3  bar  one   3.0   6.0
4  bar  two   NaN   7.0

In [32]:
pd.merge(left, right, on='key1')

key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7

In [33]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) # suffixes 用于指定附加到左右两个DataFrame对象的重叠列名上的字符串

key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7

> ### 索引上的合并：在DataFrame中的连接键位于其索引的情况

In [34]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
left1

key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5

In [35]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

group_val
a        3.5
b        7.0

In [36]:
pd.merge(left1, right1, left_on='key', right_index=True) # 传入left_index=True或right_index=True（或两个都传）以说明索引应该被用作连接键

key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0

In [37]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer') # 通过外连接方式取得并集
# 由于默认的merge方法是求取连接键的交集，可以通过外连接的方式得到其并集

key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0
5   c      5        NaN

> ### 层次化索引（索引的合并默认是多键合并）

In [39]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',\
                               'Nevada', 'Nevada'],\
                      'key2': [2000, 2001, 2002, 2001, 2002],\
                      'data': np.arange(5.)})
lefth

data    key1  key2
0   0.0    Ohio  2000
1   1.0    Ohio  2001
2   2.0    Ohio  2002
3   3.0  Nevada  2001
4   4.0  Nevada  2002

In [40]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',\
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [41]:
# 以列表的形式指明用作合并键的多个列（注意用how='outer'对重复索引值的处理）
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

data    key1  key2  event1  event2
0   0.0    Ohio  2000       4       5
0   0.0    Ohio  2000       6       7
1   1.0    Ohio  2001       8       9
2   2.0    Ohio  2002      10      11
3   3.0  Nevada  2001       0       1

In [42]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],\
         right_index=True, how='outer')

data    key1  key2  event1  event2
0   0.0    Ohio  2000     4.0     5.0
0   0.0    Ohio  2000     6.0     7.0
1   1.0    Ohio  2001     8.0     9.0
2   2.0    Ohio  2002    10.0    11.0
3   3.0  Nevada  2001     0.0     1.0
4   4.0  Nevada  2002     NaN     NaN
4   NaN  Nevada  2000     2.0     3.0

In [43]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],\ # 用合并两者的索引
                     index=['a', 'c', 'e'],\
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],\
                      index=['b', 'c', 'd', 'e'],\
                      columns=['Missouri', 'Alabama'])
print(left2)
print(right2)
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0

In [44]:
# join的实例方法 用于合并多个带有相同或相似索引的DataFrame对象，但要求没有重叠的列
left2.join(right2, how='outer') 

Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0

In [45]:
left1.join(right1, on='key')

key  value  group_val
0   a      0        3.5
1   b      1        7.0
2   a      2        3.5
3   a      3        3.5
4   b      4        7.0
5   c      5        NaN

In [46]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
print(another)
left2.join([right2, another])

   New York  Oregon
a       7.0     8.0
c       9.0    10.0
e      11.0    12.0
f      16.0    17.0


Ohio  Nevada  Missouri  Alabama  New York  Oregon
a   1.0     2.0       NaN      NaN       7.0     8.0
c   3.0     4.0       9.0     10.0       9.0    10.0
e   5.0     6.0      13.0     14.0      11.0    12.0

In [47]:
left2.join([right2, another], how='outer')

Ohio  Nevada  Missouri  Alabama  New York  Oregon
a   1.0     2.0       NaN      NaN       7.0     8.0
b   NaN     NaN       7.0      8.0       NaN     NaN
c   3.0     4.0       9.0     10.0       9.0    10.0
d   NaN     NaN      11.0     12.0       NaN     NaN
e   5.0     6.0      13.0     14.0      11.0    12.0
f   NaN     NaN       NaN      NaN      16.0    17.0

> ### 轴向连接：连接（concatenation）、绑定（binding）或堆叠（stacking）

In [48]:
arr = np.arange(12).reshape((3, 4)) # NumPy的concatenation函数可以用NumPy数组实现
print(arr)
np.concatenate([arr, arr], axis=1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [49]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [50]:
pd.concat([s1, s2, s3]) # 调用concat可以将值和索引粘合在一起

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

> #### 默认情况下，concat是在axis=0上工作的，最终产生一个新的Series。如果传入axis=1，则结果就会变成一个DataFrame（axis=1是列）

In [51]:
pd.concat([s1, s2, s3], axis=1) # concat是在axis=0上工作的，最终产生一个新的Series 
# 如果传入axis=1，则结果就会变成一个DataFrame

0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0

In [52]:
s4 = pd.concat([s1, s3])  # 传入join='inner'即可得到其交集
print(s4)
pd.concat([s1, s4], axis=1) # 使用join='inner'的操作

a    0
b    1
f    5
g    6
dtype: int64


0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6

In [53]:
pd.concat([s1, s4], axis=1, join='inner')

0  1
a  0  0
b  1  1

In [54]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']]) # 通过join_axes指定要在对应轴上使用的索引

0    1
a  0.0  0.0
c  NaN  NaN
b  1.0  1.0
e  NaN  NaN

In [55]:
# 在连接轴上创建一个层次化索引。可以使用keys参数来实现
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
print(result)
result.unstack()

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64


a    b    f    g
one    0.0  1.0  NaN  NaN
two    0.0  1.0  NaN  NaN
three  NaN  NaN  5.0  6.0

In [56]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three']) # 使用 keys参数 可以创建一个层次化索引
# 如果沿着axis=1对Series进行合并，则keys就会成为DataFrame的列头

one  two  three
a  0.0  NaN    NaN
b  1.0  NaN    NaN
c  NaN  2.0    NaN
d  NaN  3.0    NaN
e  NaN  4.0    NaN
f  NaN  NaN    5.0
g  NaN  NaN    6.0

In [57]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],\ 
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],\
                   columns=['three', 'four'])
print(df1)
print(df2)
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [58]:
pd.concat({'level1': df1, 'level2': df2}, axis=1) # 如果传入的不是列表而是一个字典，则字典的键就会被当做keys选项的值

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [59]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],\ # 用names参数命名创建的轴级别
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [60]:
# DataFrame的行索引不包含任何相关数据
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
print(df1)
df2

          a         b         c         d
0  1.246435  1.007189 -1.296221  0.274992
1  0.228913  1.352917  0.886429 -2.001637
2 -0.371843  1.669025 -0.438570 -0.539741


b         d         a
0  0.476985  3.248944 -1.021228
1 -0.577087  0.124121  0.302614

In [61]:
pd.concat([df1, df2], ignore_index=True)  # 设定ignore_index=True

a         b         c         d
0  1.246435  1.007189 -1.296221  0.274992
1  0.228913  1.352917  0.886429 -2.001637
2 -0.371843  1.669025 -0.438570 -0.539741
3 -1.021228  0.476985       NaN  3.248944
4  0.302614 -0.577087       NaN  0.124121

> ### 合并重叠数据：存在疑惑？

> #### Series的combine_first方法

> #### **用传递对象中的数据为调用对象的缺失数据**

In [62]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print(a)
print(b)
np.where(pd.isnull(a), b, a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64


array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [63]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [64]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
print(df1)
print(df2)
df1.combine_first(df2)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14
     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


a    b     c
0  1.0  NaN   2.0
1  4.0  2.0   6.0
2  5.0  4.0  10.0
3  3.0  6.0  14.0
4  7.0  8.0   NaN

### 2.3 重塑 和 轴向旋转

> #### 用于重新排列表格型数据的基础运算

> #### 重塑（reshape）运算

> #### 轴向旋转（pivot）运算

> ### 重塑层次化索引

> #### 层次化索引为DataFrame数据的重排任务提供了:

> #### stack：将数据的列“旋转”为行

> #### unstack：将数据的行“旋转”为列

In [65]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),\
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),\
                    columns=pd.Index(['one', 'two', 'three'],\
                    name='number'))  # 行列索引均为字符串数组
data

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5

In [66]:
# 对该数据使用stack方法即可将列转换为行，得到一个Series
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [67]:
# 对于一个层次化索引的Series，你可以用unstack将其重排为一个DataFrame
result.unstack()

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5

#### 默认情况下，unstack操作的是最内层（stack也是如此）。

#### 传入分层级别的编号或名称即可对其它级别进行unstack操作

In [68]:
result.unstack(0)

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5

In [69]:
result.unstack('state')

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5

#### 如果不是所有的级别值都能在各分组中找到的话，则unstack操作可能会引入缺失数据

In [70]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
print(data2)
data2.unstack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64


a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0

In [71]:
data2.unstack() #stack默认会滤除缺失数据，因此该运算是可逆的 

a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0

In [72]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [73]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [74]:
# 在对DataFrame进行unstack操作时，作为旋转轴的级别将会成为结果中的最低级别
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
print(df)
df.unstack('state')

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10


side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [75]:
df.unstack('state').stack('side') # 调用stack，指明轴名

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

> ### 将“长格式”旋转为“宽格式”

> #### 时间序列规整和数据清洗

In [76]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

year  quarter   realgdp  realcons  realinv  realgovt  realdpi    cpi  \
0  1959.0      1.0  2710.349    1707.4  286.898   470.045   1886.9  28.98   
1  1959.0      2.0  2778.801    1733.7  310.859   481.301   1919.7  29.15   
2  1959.0      3.0  2775.488    1751.8  289.226   491.260   1916.4  29.35   
3  1959.0      4.0  2785.204    1753.7  299.356   484.052   1931.3  29.37   
4  1960.0      1.0  2847.699    1770.5  331.722   462.199   1955.5  29.54   

      m1  tbilrate  unemp      pop  infl  realint  
0  139.7      2.82    5.8  177.146  0.00     0.00  
1  141.7      3.08    5.1  177.830  2.34     0.74  
2  140.5      3.82    5.3  178.657  2.74     1.09  
3  140.0      4.33    5.6  179.386  0.27     4.06  
4  139.6      3.50    5.2  180.007  2.31     1.19

In [77]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,\
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
print(periods)
print(columns)

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')
Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')


In [78]:
data = data.reindex(columns=columns)
print(data)

item    realgdp  infl  unemp
0      2710.349  0.00    5.8
1      2778.801  2.34    5.1
2      2775.488  2.74    5.3
3      2785.204  0.27    5.6
4      2847.699  2.31    5.2
5      2834.390  0.14    5.2
6      2839.022  2.70    5.6
7      2802.616  1.21    6.3
8      2819.264 -0.40    6.8
9      2872.005  1.47    7.0
..          ...   ...    ...
193   13203.977  2.75    4.5
194   13321.109  3.45    4.7
195   13391.249  6.38    4.8
196   13366.865  2.82    4.9
197   13415.266  8.53    5.4
198   13324.600 -3.16    6.0
199   13141.920 -8.79    6.9
200   12925.410  0.94    8.1
201   12901.504  3.37    9.2
202   12990.341  3.56    9.6

[203 rows x 3 columns]


In [79]:
data.index = periods.to_timestamp('D', 'end')

In [80]:
# 多个时间序列（其键是date和item）的长格式
# 表中的每行代表一次观察
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [81]:
ldata[:10]

date     item     value
0 1959-03-31  realgdp  2710.349
1 1959-03-31     infl     0.000
2 1959-03-31    unemp     5.800
3 1959-06-30  realgdp  2778.801
4 1959-06-30     infl     2.340
5 1959-06-30    unemp     5.100
6 1959-09-30  realgdp  2775.488
7 1959-09-30     infl     2.740
8 1959-09-30    unemp     5.300
9 1959-12-31  realgdp  2785.204

In [82]:
# date和item通常就是主键
# 而DataFrame，不同的item值分别形成一列，date列中的时间戳则用作索引。DataFrame的pivot方法完全可以实现这个转换
pivoted = ldata.pivot('date', 'item', 'value')
pivoted
# 前两个传递的值分别用作行和列索引，最后一个可选值则是用于填充DataFrame的数据列

item        infl    realgdp  unemp
date                              
1959-03-31  0.00   2710.349    5.8
1959-06-30  2.34   2778.801    5.1
1959-09-30  2.74   2775.488    5.3
1959-12-31  0.27   2785.204    5.6
1960-03-31  2.31   2847.699    5.2
1960-06-30  0.14   2834.390    5.2
1960-09-30  2.70   2839.022    5.6
1960-12-31  1.21   2802.616    6.3
1961-03-31 -0.40   2819.264    6.8
1961-06-30  1.47   2872.005    7.0
...          ...        ...    ...
2007-06-30  2.75  13203.977    4.5
2007-09-30  3.45  13321.109    4.7
2007-12-31  6.38  13391.249    4.8
2008-03-31  2.82  13366.865    4.9
2008-06-30  8.53  13415.266    5.4
2008-09-30 -3.16  13324.600    6.0
2008-12-31 -8.79  13141.920    6.9
2009-03-31  0.94  12925.410    8.1
2009-06-30  3.37  12901.504    9.2
2009-09-30  3.56  12990.341    9.6

[203 rows x 3 columns]

In [83]:
# 需要处理两个同时重塑的数据列
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

date     item     value    value2
0 1959-03-31  realgdp  2710.349  0.523772
1 1959-03-31     infl     0.000  0.000940
2 1959-03-31    unemp     5.800  1.343810
3 1959-06-30  realgdp  2778.801 -0.713544
4 1959-06-30     infl     2.340 -0.831154
5 1959-06-30    unemp     5.100 -2.370232
6 1959-09-30  realgdp  2775.488 -1.860761
7 1959-09-30     infl     2.740 -0.860757
8 1959-09-30    unemp     5.300  0.560145
9 1959-12-31  realgdp  2785.204 -1.265934

In [84]:
# 如果忽略最后一个参数，得到的DataFrame就会带有层次化的列
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.000940  0.523772  1.343810
1959-06-30  2.34  2778.801   5.1 -0.831154 -0.713544 -2.370232
1959-09-30  2.74  2775.488   5.3 -0.860757 -1.860761  0.560145
1959-12-31  0.27  2785.204   5.6  0.119827 -1.265934 -1.063512
1960-03-31  2.31  2847.699   5.2 -2.359419  0.332883 -0.199543

In [85]:
pivoted['value'][:5]

item        infl   realgdp  unemp
date                             
1959-03-31  0.00  2710.349    5.8
1959-06-30  2.34  2778.801    5.1
1959-09-30  2.74  2775.488    5.3
1959-12-31  0.27  2785.204    5.6
1960-03-31  2.31  2847.699    5.2

In [86]:
# pivot其实就是用set_index创建层次化索引，再用unstack重塑
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.000940  0.523772  1.343810
1959-06-30  2.34  2778.801   5.1 -0.831154 -0.713544 -2.370232
1959-09-30  2.74  2775.488   5.3 -0.860757 -1.860761  0.560145
1959-12-31  0.27  2785.204   5.6  0.119827 -1.265934 -1.063512
1960-03-31  2.31  2847.699   5.2 -2.359419  0.332883 -0.199543
1960-06-30  0.14  2834.390   5.2 -0.970736 -1.541996 -1.307030
1960-09-30  2.70  2839.022   5.6  0.377984  0.286350 -0.753887

> ### 将“宽格式”旋转为“长格式”

> #### 旋转DataFrame的逆运算是pandas.melt。它不是将一列转换到多个新的DataFrame，而是合并多个列成为一个，产生一个比输入长的DataFrame

In [87]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

A  B  C  key
0  1  4  7  foo
1  2  5  8  bar
2  3  6  9  baz

In [88]:
# key列可能是分组指标，其它的列是数据值。当使用pandas.melt，我们必须指明列的分组指标
melted = pd.melt(df, ['key']) # 可以使用key作为唯一的分组指标
melted

key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6
6  foo        C      7
7  bar        C      8
8  baz        C      9

In [89]:
# 使用pivot，可以重塑回源数据
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable  A  B  C
key              
bar       2  5  8
baz       3  6  9
foo       1  4  7

In [90]:
# pivot的结果从列创建了一个索引，用作行标签，我们可以使用reset_index将数据移回列
reshaped.reset_index()

variable  key  A  B  C
0         bar  2  5  8
1         baz  3  6  9
2         foo  1  4  7

In [91]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B']) # 也可以指定列的子集，作为值的列

key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6

In [92]:
pd.melt(df, value_vars=['A', 'B', 'C']) # 不用分组指标的情况

variable  value
0        A      1
1        A      2
2        A      3
3        B      4
4        B      5
5        B      6
6        C      7
7        C      8
8        C      9

In [93]:
pd.melt(df, value_vars=['key', 'A', 'B'])

variable value
0      key   foo
1      key   bar
2      key   baz
3        A     1
4        A     2
5        A     3
6        B     4
7        B     5
8        B     6

### Conclusion

> ### pandas的数据导入、清洗和重塑过程

> ### 重难点在于重塑过程


> ### 与前期章节的相关内容进行衔接